In [1]:
import pandas as pd
import numpy as np
import re
import os
import datetime as dt

In [2]:
df_nidec = pd.read_excel("General Claim Returns 51-motors.xlsx")
df_supp  = pd.read_excel("Nidec_supplier.xlsx",sheet_name='Data')

In [3]:
def rep(x):
    if x.startswith('51-102497-'):
        return '51-101920-00'
    elif x.startswith('51-103823-'):
        return '51-101984-00'
    elif x.startswith('51-104337-'):
        return '51-101879-00'
    elif x.startswith('51-104338-'):
        return '51-101880-00'
    elif x.startswith('51-104339-'):
        return '51-101920-00'
    elif x.startswith('51-104340-'):
        return '51-101984-00'
    else:
        return x
    
df_new = df_nidec.copy() 
df_nidec['Entered Failed Part'] = df_nidec['Entered Failed Part'].apply(rep)
df_nidec[df_nidec['Entered Failed Part'].isin(['51-101920-00','51-101984-00','51-101879-00','51-101880-00','51-101920-00','51-101984-00'])]['Entered Failed Part'].value_counts()

51-101920-00    20537
51-101880-00    19310
51-101879-00     4239
51-101984-00     2553
Name: Entered Failed Part, dtype: int64

In [20]:
df_nidec['part_family']= df_nidec['Entered Failed Part'].apply(lambda x: "-".join(x.split('-',2)[:2]))
df_nidec['part_family'] = df_nidec['part_family'].replace({'51-102497':'51-101920' , '51-103823': '51-101984',\
                                                      '51-104337':'51-101879','51-104338':'51-101880',\
                                                        '51-104339':'51-101920','51-104340':'51-101984'})

df_nidec[df_nidec.part_family.isin(['51-101920','51-101984','51-101879','51-101880'])]['part_family'].value_counts()
df_nidec["num_days"] = (pd.to_datetime(df_nidec['Claim Transaction Date'])-pd.to_datetime(df_nidec['Failed Unit Production Date'])).dt.days
nidec_filt = df_nidec[df_nidec['DIFFERENCE']=="KEEP"]
nidec_filt.to_excel("FILTERED_Nidec.xlsx",index=False)
nidec_filt['num_claims'] = 1
nidec_filt['Production_year'] = pd.to_datetime(nidec_filt['Failed Unit Production Date']).dt.year


C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_4872\2718843099.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nidec_filt['num_claims'] = 1
C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_4872\2718843099.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nidec_filt['Production_year'] = pd.to_datetime(nidec_filt['Failed Unit Production Date']).dt.year


In [21]:
nidec_grp = nidec_filt.groupby(['TRANSACTION YEAR','Entered Failed Part'])[['num_claims','Reserve Cost']].sum().reset_index()
nidec_grp = nidec_grp.rename(columns={'TRANSACTION YEAR':'Year'})
nidec_grp.head()

,Year,Entered Failed Part,num_claims,Reserve Cost
0,2015,51-100837-04,3,140.55825
1,2015,51-100998-05,10,306.78250
2,2015,51-100998-08,11,405.78725
3,2015,51-100998-12,2,74.45600
4,2015,51-100998-13,37,1235.21725


In [22]:
df_supp
# Unpivot the DataFrame
df_unpivoted = pd.melt(df_supp, id_vars=['Part Number'], var_name='Year', value_name='Value')
 # Drop rows where 'Value' is None 
df_unpivoted = df_unpivoted.dropna(subset=['Value']) 
df_unpivoted =df_unpivoted.rename(columns= {'Part Number':'Entered Failed Part','Value':'shipped_units'})
# df_unpivoted['Value'] = df_unpivoted['Value'].round(decimals=0).astype(object)
df_unpivoted

,Entered Failed Part,Year,shipped_units
4,51-100838-03,2015,112.00
5,51-100999-02,2015,372.00
6,51-100999-24,2015,28643.00
7,51-102008-01,2015,540.00
8,51-102008-02,2015,1226.00
...,...,...,...
1134,51-42537-01,2023,10.00
1137,51-42537-05,2023,2853.00
1138,51-42537-06,2023,8963.00
1139,51-42537-07,2023,33.00


In [23]:
final_grped = df_unpivoted.merge(nidec_grp,on=['Year','Entered Failed Part'],how='left')
# final_grped[final_grped['Entered Failed Part']=='51-100998-24']
final_grped.to_excel('Final_data_NIDEC.xlsx',index=False)

In [24]:
final_grped['Rolling_Avg_Shipped_Units'] = final_grped.groupby('Entered Failed Part')['shipped_units'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True) 
# Display the DataFrame with rolling average 
final_grped[['Entered Failed Part', 'Year', 'shipped_units', 'num_claims', 'Rolling_Avg_Shipped_Units']]
final_grped['Rolling_Avg_Shipped_Units']  = final_grped['Rolling_Avg_Shipped_Units'].round(decimals=0)
final_grped['Reserve Cost']  = final_grped['Reserve Cost'].round(decimals=2)



In [25]:
final_grped['rate'] = round((final_grped['num_claims']/final_grped['Rolling_Avg_Shipped_Units'])*100,2)
final_grped

,Entered Failed Part,Year,shipped_units,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units,rate
0,51-100838-03,2015,112.00,NaN,NaN,112.0,NaN
1,51-100999-02,2015,372.00,NaN,NaN,372.0,NaN
2,51-100999-24,2015,28643.00,8.0,314.47,28643.0,0.03
3,51-102008-01,2015,540.00,1.0,22.02,540.0,0.19
4,51-102008-02,2015,1226.00,1.0,35.16,1226.0,0.08
...,...,...,...,...,...,...,...
622,51-42537-01,2023,10.00,3.0,480.37,451.0,0.67
623,51-42537-05,2023,2853.00,21.0,3122.08,3176.0,0.66
624,51-42537-06,2023,8963.00,24.0,3898.94,5153.0,0.47
625,51-42537-07,2023,33.00,NaN,NaN,28.0,NaN


In [26]:
nidec_filt['num_claims']=1
df_nidec_fam = nidec_filt.groupby(['TRANSACTION YEAR','part_family'])[['num_claims','Reserve Cost']].sum().reset_index()
df_nidec_fam = df_nidec_fam.rename(columns={'TRANSACTION YEAR':'Year'})
# Unpivot the DataFrame
df_unpivoted_fam = pd.melt(df_supp, id_vars=['Part Number'], var_name='Year', value_name='Value')
 # Drop rows where 'Value' is None 
df_unpivoted_fam = df_unpivoted_fam.dropna(subset=['Value']) 
df_unpivoted_fam =df_unpivoted_fam.rename(columns= {'Part Number':'Entered Failed Part','Value':'shipped_units'})
df_unpivoted_fam['part_family'] =df_unpivoted_fam['Entered Failed Part'].apply(lambda x: "-".join(x.split('-',2)[:2]))
df_unpivoted_family = df_unpivoted_fam.groupby(['part_family','Year'])['shipped_units'].sum().reset_index()
df_final_grped_family = df_unpivoted_family.merge(df_nidec_fam,on=['Year','part_family'],how='left')
# df_final_grped_family[df_final_grped_family['Entered Failed Part']=='51-100998-24']
df_final_grped_family.to_excel('Final_data_NIDEC_family.xlsx',index=False)
df_final_grped_family['Rolling_Avg_Shipped_Units'] = df_final_grped_family.groupby('part_family')['shipped_units'].rolling(window=6, min_periods=1).mean().reset_index(level=0, drop=True) 
# Display the DataFrame with rolling average 
df_final_grped_family[['part_family', 'Year', 'shipped_units', 'num_claims', 'Rolling_Avg_Shipped_Units']]
df_final_grped_family['Rolling_Avg_Shipped_Units']  = df_final_grped_family['Rolling_Avg_Shipped_Units'].round(decimals=0)
df_final_grped_family['Reserve Cost']  = df_final_grped_family['Reserve Cost'].round(decimals=2)
df_final_grped_family['rate'] = round((df_final_grped_family['num_claims']/df_final_grped_family['Rolling_Avg_Shipped_Units'])*100,2)
# df_final_grped['rate'] = round((df_final_grped['num_claims']/df_final_grped['Rolling_Avg_Shipped_Units'])*100,2)
df_final_grped_family.to_excel("SCORE_CARD_NIDEC_FAMILY.xlsx",index=False)

df_final_grped_family.head()

# df_unpivoted
# df_reg_grp.head()


C:\Users\aamir.rafi\AppData\Local\Temp\ipykernel_4872\2866334298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nidec_filt['num_claims']=1


,part_family,Year,shipped_units,num_claims,Reserve Cost,Rolling_Avg_Shipped_Units,rate
0,08216216,2021,600.0,NaN,NaN,600.0,NaN
1,08216216,2022,1799.0,NaN,NaN,1200.0,NaN
2,08216216,2023,2000.0,NaN,NaN,1466.0,NaN
3,47-109389,2023,180.0,NaN,NaN,180.0,NaN
4,47-109390,2023,450.0,NaN,NaN,450.0,NaN


In [28]:
nidec_filt['DIFFERENCE'].unique()

array(['KEEP'], dtype=object)

In [30]:
df_nidec['Claims_count'] =1
df_2023_nidec = df_nidec[df_nidec['TRANSACTION YEAR']==2023].groupby(['TRANSACTION YEAR','part_family'])[['Claims_count','Reserve Cost']].sum().reset_index()
df_2023_nidec = df_2023_nidec.rename(columns={'Reserve Cost':'Reserve_cost_2023','TRANSACTION YEAR':'Year'})
df_2023_nidec.head()
df_final_grped_family.merge(df_2023_nidec,on=['part_family','Year'],how='left').to_excel('SCORE_CARD_NIDEC_FAMILY_UPDATED.xlsx',index=False)


In [19]:
df_nidec.columns

Index(['Claim Number', 'Failed Unit Product Class',
       'Failed Unit Customer Name', 'Installer Name',
       'Entered Failed Unit Number', 'Entered Failed Unit Serial',
       'Entered Repl Unit', 'Entered Repl Unit Serial', 'Entered Failed Part',
       'Entered Failed Part Serial Number', 'Entered Repl Part',
       'Claim Transaction Date', 'TRANSACTION YEAR',
       'Failed Unit Production Date', 'Claim Type Name', 'QA Reason Code Name',
       'QA Reason Code Comments', 'Price', 'Labor Amount', 'Reserve Cost',
       'Failed Part Product Name', 'DIFFERENCE', 'part_family', 'num_days',
       'num_claims'],
      dtype='object')